In [1]:
import pandas as pd
import numpy as np

from keras.models import load_model

Using TensorFlow backend.


In [2]:
FILE_NAME = 'data.csv'
MODEL_QUESTIONS_FILE = 'model_questions.h5'
MODEL_ANSWER_FILE = 'model_answer.h5'
MODELS_DATA_FILE = 'models_data.npy'

df = pd.read_csv(FILE_NAME, index_col=0)
idx2col = {i:col for i, col in enumerate(df.columns)}
idx2row = {i:row for i, row in enumerate(df.index)}
idx2answer = {1: 'Si', 0.5: 'Creo que si', 0: 'No se', -0.5: 'Creo que no', -1: 'No'}
data = df.values
n_questions = len(idx2col)
n_plates = len(idx2row)

print('Data information:')
print('Number of plates: %d' % n_plates)
print('Number of questions: %d' % n_questions)
print('\nloading models...', end='\r')
model_questions = load_model(MODEL_QUESTIONS_FILE)
model_answers = load_model(MODEL_ANSWER_FILE)
temp = np.load(MODELS_DATA_FILE)
model_data = {}
model_data['max_q'] = temp[0]
model_data['first_q'] = temp[1]
print('Models loaded.')

Data information:
Number of plates: 41
Number of questions: 23

Models loaded....


In [3]:
# Model utils
def build_input(path, data):
    out_a = np.zeros_like(data)
    out_b = np.zeros_like(data)
    out_a[:len(path)] = np.asarray(path)
    out_a = out_a.reshape([1, -1])
    out_b[:len(path)] = data[path]
    out_b = out_b.reshape([1, -1])
    
    return [out_a, out_b]

def get_output(y, top=5):
    questions, choice = y
    out_type = 'question' if np.argmax(choice) == 0 else 'answer'
    top_idx = questions.argsort()[0][::-1][:top]
    top_prb = questions[0, top_idx]
    
    return out_type, top_idx, top_prb

def ask_question(q):
    q += '\n(1) No'
    q += '\n(2) No se'
    q += '\n(3) Si\n'
    out = {1:-1, 2:0, 3:1}
    
    while True:
        try:
            x = int(input(q))
        except:
            None
        else:
            if 1 <= x <= 3:
                return out[x]
            elif x == 4:
                raise 'lalalal'

In [4]:
model_data['max_q']

14

In [6]:
model_data['max_q'] = 12
model_data['max_q']

12

In [18]:
# Ask questions
test_path = []
i = 0
pred_type = 'question'
plate_data = np.zeros_like(data[0])

while pred_type != 'answer':
    if i == 0:
        pred_type = 'question'
        pred_val = model_data['first_q']
    else:
        pred = model_questions.predict(build_input(test_path, plate_data))
        limit = n_questions
        pred_type, pred_vals, pred_probs = get_output(pred, limit)
        
        for j in range(limit):
            if pred_vals[j] in test_path:
                continue
            else:
                pred_val = pred_vals[j]
                break
        
        answer = np.zeros_like(plate_data)
        idxs = np.unique([i for i in test_path if i is not None])
        answer[idxs] = plate_data[idxs]
        pred2 = model_answers.predict(answer.reshape([1,-1]))
        t_answers = np.sum(np.round(pred2[0], 4) > 0)
        t_confidence = np.any(pred[1] > .999)
        
        if t_confidence and t_answers <= 5:
            final_plates = np.argsort(pred2[0])[::-1][:t_answers]
            break
   
    if i < model_data['max_q']:
        print('\nQuestion #%d' % (i + 1))
        a = ask_question(idx2col[pred_val])
        plate_data[pred_val] = a
        print(idx2answer[a])
        pred_type = 'question'
    else:
        final_plates = np.argsort(pred2[0])[::-1][:5]
        break
        
    test_path.append(pred_val)
    i += 1
    
n_guess = len(final_plates)

for j in range(n_guess):
    if j == n_guess-1:
        final = final_plates[j]
        break
    
    print('\nQuestion #%d' % (i + 1))
    i += 1
    a = ask_question('Es %s?' % idx2row[final_plates[j]])
    if a == 1:
        final = final_plates[j]
        break

print('\nPLATE = ' + idx2row[final])


Question #1
Lleva cebolla?
(1) No
(2) No se
(3) Si
1
No

Question #2
Lleva fideos?
(1) No
(2) No se
(3) Si
1
No

Question #3
Lleva carne de res?
(1) No
(2) No se
(3) Si
1
No

Question #4
Es comida marina?
(1) No
(2) No se
(3) Si
1
No

Question #5
Es frio?
(1) No
(2) No se
(3) Si
1
No

Question #6
Lleva pollo/gallina?
(1) No
(2) No se
(3) Si
3
Si

Question #7
Lleva chicha de jora?
(1) No
(2) No se
(3) Si
1
No

Question #8
Lleva pescado?
(1) No
(2) No se
(3) Si
1
No

Question #9
Es liquido/sopa?
(1) No
(2) No se
(3) Si
1
No

Question #10
Lleva zapallo?
(1) No
(2) No se
(3) Si
1
No

Question #11
Lleva arroz?
(1) No
(2) No se
(3) Si
3
Si

Question #12
Lleva chancho?
(1) No
(2) No se
(3) Si
1
No

Question #13
Es Arroz con pollo peruano?
(1) No
(2) No se
(3) Si
1

Question #14
Es Arroz chaufa?
(1) No
(2) No se
(3) Si
3

PLATE = Arroz chaufa
